# Première de création du model

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [5]:
X = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Audio/X_(clean_only).csv')
X=X.drop(X.columns[0], axis=1)
X=tf.convert_to_tensor(X, dtype=tf.complex64)


In [6]:
y = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Audio/y_5Gb.csv')

y=y.drop(y.columns[0], axis=1)
y=tf.convert_to_tensor(y, dtype=tf.complex64)

# Remove target
#X = clean.copy()
preprocessor = make_column_transformer(
    (StandardScaler(),
     make_column_selector(dtype_include=np.number)),
    (OneHotEncoder(sparse=False),
     make_column_selector(dtype_include=object)),
)
#X = preprocessor.fit_transform(X)
#y = np.log(y) # log transform target instead of standardizing

input_shape = [X.shape[1]]

print("Input shape: {}".format(input_shape))

Input shape: [1]


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importation des bibliothèques nécessaires

### Configuration du model 

In [8]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
          layers.BatchNormalization(input_shape=input_shape),
          layers.Dense(units=2, activation='relu'),
          layers.BatchNormalization(),
          layers.Dense(units=2, activation='relu'),
          layers.BatchNormalization(),
          layers.Dense(units=2, activation='relu'),
          layers.Dense(units=2,activation='sigmoid')
          #layers.Dense(units=2, activation='relu'),
          #layers.Dense(units=2, activation='relu'),
          #layers.Dense(units=2, activation='relu'),


])

In [9]:
#opt = keras.optimizers.Adam(learning_rate=0.00009)
model.compile(
    optimizer = 'Adam',
    loss = 'mae',
)

### Entrainement du modele

In [ ]:
history = model.fit(
    X, y,
    batch_size=256,
    epochs=10,
)
# Start the plot at epoch 10
history_df = pd.DataFrame(history.history)
history_df.loc[0:, ['loss']].plot()

In [ ]:
y_predict=model.predict(X)

In [17]:
import os
Audio_dir='/content/drive/MyDrive/Colab Notebooks/Audio'
os.chdir(Audio_dir)
import librosa
import IPython.display as ipd
ipd.Audio(librosa.core.istft(y_predict),rate=44100)
pd.DataFrame(y_predict).to_csv("y_predicted_175.csv")


In [ ]:
#Mise en place d'un early stopping : 

# Certainement à utiliser plus tard...
#from tensorflow.keras.callbacks import EarlyStopping

#early_stopping = EarlyStopping(
#    min_delta=0.001, # minimium amount of change to count as an improvement
#    patience=20, # how many epochs to wait before stopping
#    restore_best_weights=True,
#)